In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

import requests
import SimpleITK as sitk
import pathlib

from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
import alphashape
from alpha_shapes.boundary import Boundary, get_boundaries

In [ ]:
ClusterAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")
def add_class(BiasDF, ClusterAnn):
    for cluster, row in BiasDF.iterrows():
        BiasDF.loc[cluster, "class_id_label"] = ClusterAnn.loc[cluster, "class_id_label"]
        BiasDF.loc[cluster, "CCF_broad.freq"] = ClusterAnn.loc[cluster, "CCF_broad.freq"]
        BiasDF.loc[cluster, "CCF_acronym.freq"] = ClusterAnn.loc[cluster, "CCF_acronym.freq"]
        BiasDF.loc[cluster, "v3.size"] = ClusterAnn.loc[cluster, "v3.size"]
        BiasDF.loc[cluster, "v2.size"] = ClusterAnn.loc[cluster, "v2.size"]
    return BiasDF

In [ ]:
MERFISH_Exp_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Cell_Mean_DF.UMI.csv",index_col=0)
MERFISH_Exp_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Vol_Mean_DF.UMI.csv", index_col=0)
#MERFISH_Exp_Cell2 = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv")

In [ ]:
head = MERFISH_Exp_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Exp_Cell.columns = new_head
MERFISH_Exp_Vol.columns = new_head

In [ ]:
ISH_Exp = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/JW_ExpMat.LogMean.0418.csv", index_col=0)

In [ ]:
ISH_Entrez = ISH_Exp.index.values

In [ ]:
ISH_Symbol = np.array([Entrez2Symbol.get(x, ".") for x in ISH_Entrez])
ISH_Symbol = ISH_Symbol[ISH_Symbol!="."]

In [ ]:
len(ISH_Symbol)

In [ ]:
MERFISH_Entrez = MERFISH_Exp_Cell.index.values
MERFISH_Symbol = np.array([Entrez2Symbol.get(x, ".") for x in ISH_Entrez])
MERFISH_Symbol = MERFISH_Symbol[MERFISH_Symbol!="."]
len(MERFISH_Symbol)

In [ ]:
ISH_STRs = ISH_Exp.columns.values
MERFISH_STRs = MERFISH_Exp_Cell.columns.values
Shared_STRs = sorted(list(set(ISH_STRs).intersection(set(MERFISH_STRs))))
Shared_Genes = sorted(list(set(MERFISH_Exp_Cell.index.values).intersection(set(ISH_Exp.index.values))))

In [ ]:
count = 0
for g in Shared_Genes:
    symbol = Entrez2Symbol.get(g, ".")
    if symbol != ".":
        count += 1 
print(count)

In [ ]:
# Total Exp for each STR
dat1 = []
dat2 = []
dat3 = []
for STR in Shared_STRs:
    dat1.append(ISH_Exp[STR].sum())
    dat2.append(MERFISH_Exp_Cell[STR].sum())
    dat3.append(MERFISH_Exp_Vol[STR].sum())

In [ ]:
df = pd.DataFrame(data={"STR":Shared_STRs, "ISH":dat1, "MERFISH_Cell":dat2,  "MERFISH_Vol":dat3})
df['ISH_rank'] = df['ISH'].rank()
df['MERFISH_Cell_rank'] = df['MERFISH_Cell'].rank()
df['MERFISH_Vol_rank'] = df['MERFISH_Vol'].rank()
df = df.set_index("STR")

In [ ]:
str2reg = STR2Region()

In [ ]:
# Annotate Region
for i, row in df.iterrows():
    df.loc[i, "Region"] = str2reg[i]

In [ ]:
Regions = list(set(df["Region"]))
Regions

In [ ]:
#plt.figure(dpi=40)
"""
for region in Regions:
    plt.figure(dpi=120)
    tmpdf = df[df["Region"]==region]
    plt.scatter(df["MERFISH_Vol_rank"], df["ISH_rank"], color="grey", alpha=0.3)
    plt.scatter(tmpdf["MERFISH_Vol_rank"], tmpdf["ISH_rank"], color="red")
    plt.title(region)
    plt.xlabel("MERFISH Vol Total Exp")
    plt.ylabel("ISH Total EXP")
    plt.show()
"""
#plt.legend()

In [ ]:
num_regions = len(Regions)
num_cols = 3  # Number of columns
num_rows = (num_regions + num_cols - 1) // num_cols  # Calculate number of rows

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 10), dpi=480, sharex=True, sharey=True)

# Flatten the axes array if it's not already flat
if not isinstance(axes, np.ndarray):
    axes = np.array(axes)

# Iterate through each region and plot on a subplot
for i, region in enumerate(Regions):
    # Calculate row and column indices for the subplot
    row_idx = i // num_cols
    col_idx = i % num_cols
    
    # Filter data for the current region
    tmpdf = df[df["Region"] == region]
    
    # Plot data on the subplot
    axes[row_idx, col_idx].scatter(tmpdf["MERFISH_Cell_rank"], tmpdf["ISH_rank"], color="red", s=3)
    axes[row_idx, col_idx].scatter(df["MERFISH_Cell_rank"], df["ISH_rank"], color="grey", alpha=0.3, s=3)
    axes[row_idx, col_idx].set_title(region)

# Add common labels and title
fig.suptitle("Scatter Plot of MERFISH Rank vs ISH Rank for Different Regions")
fig.text(0.5, 0.04, "MERFISH Rank", ha="center")
fig.text(0.04, 0.5, "ISH Rank", va="center", rotation="vertical")

# Adjust layout
plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])
#plt.legend()

### Test each individual gene's correlation cross structures

In [ ]:
g_corr = []
for g in Shared_Genes:
    dat1 = MERFISH_Exp_Cell.loc[g, Shared_STRs]
    dat2 = ISH_Exp.loc[g, Shared_STRs]
    r,p = spearmanr(dat1, dat2, nan_policy="omit")
    g_corr.append(r)

In [ ]:
g_corr = np.array(g_corr)

In [ ]:
g_corr_new = g_corr[g_corr<0]
g_corr_new = np.concatenate((g_corr_new, -g_corr_new))

In [ ]:
sns.kdeplot(g_corr)
sns.kdeplot(g_corr_new)
plt.xlabel("Corr corss STRs")

In [ ]:
g_corr_df = pd.DataFrame(data={"Genes":Shared_Genes, "Corr":g_corr})
g_corr_df = g_corr_df.sort_values("Corr", ascending=False)
g_corr_df = g_corr_df.set_index("Genes")

In [ ]:
for i, row in g_corr_df.iterrows():
    g_corr_df.loc[i, "Symbol"] = Entrez2Symbol.get(float(i), ".")

In [ ]:
g_corr_df.head(2)

In [ ]:
#g_corr_df.to_csv("")

In [ ]:
g_corr_df_w_Symbol = g_corr_df[g_corr_df["Symbol"]!="."]
g_corr_df_w_Symbol.shape

In [ ]:
g_corr_df.to_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.csv")

In [ ]:
HighlyCorrG = g_corr_df_w_Symbol[g_corr_df_w_Symbol["Corr"]>0.5]
HighlyCorrG.shape

In [ ]:
ISH_Exp_HighCorr = ISH_Exp.loc[HighlyCorrG.index.values, Shared_STRs]
Merfish_Exp_HighCorr = MERFISH_Exp_Cell.loc[HighlyCorrG.index.values, Shared_STRs]

In [ ]:
ISH_Exp_HighCorr

In [ ]:
dat1 = []
dat2 = []
for STR in Shared_STRs:
    dat1.append(ISH_Exp_HighCorr[STR].sum())
    dat2.append(Merfish_Exp_HighCorr[STR].sum())
    
df = pd.DataFrame(data={"STR":Shared_STRs, "ISH":dat1, "MERFISH_Cell":dat2})
df['ISH_rank'] = df['ISH'].rank()
df['MERFISH_Cell_rank'] = df['MERFISH_Cell'].rank()
df = df.set_index("STR")
for i, row in df.iterrows():
    df.loc[i, "Region"] = str2reg[i]

In [ ]:
# Calculate the number of rows and columns for subplots
num_regions = len(Regions)
num_cols = 3  # Number of columns
num_rows = (num_regions + num_cols - 1) // num_cols  # Calculate number of rows

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 10), dpi=480, sharex=True, sharey=True)

# Flatten the axes array if it's not already flat
if not isinstance(axes, np.ndarray):
    axes = np.array(axes)

# Iterate through each region and plot on a subplot
for i, region in enumerate(Regions):
    # Calculate row and column indices for the subplot
    row_idx = i // num_cols
    col_idx = i % num_cols
    
    # Filter data for the current region
    tmpdf = df[df["Region"] == region]
    
    # Plot data on the subplot
    axes[row_idx, col_idx].scatter(tmpdf["MERFISH_Cell_rank"], tmpdf["ISH_rank"], color="red", s=3)
    axes[row_idx, col_idx].scatter(df["MERFISH_Cell_rank"], df["ISH_rank"], color="grey", alpha=0.3, s=3)
    axes[row_idx, col_idx].set_title(region)

# Add common labels and title
fig.suptitle("Scatter Plot of MERFISH Rank vs ISH Rank for Different Regions")
fig.text(0.5, 0.04, "MERFISH Rank", ha="center")
fig.text(0.04, 0.5, "ISH Rank", va="center", rotation="vertical")

# Adjust layout
plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])

# Show the plot
plt.show()

In [ ]:
tmp = g_corr_df_w_Symbol[g_corr_df_w_Symbol["Corr"]<-0.25]

In [ ]:
for g in tmp["Symbol"].values:
    print(g)

In [ ]:
g_corr_df = g_corr_df[g_corr_df["Corr"]>-1]

In [ ]:
g_corr_df.head(50)

In [ ]:
len(Entrez2Symbol)

### Correlation Distribution for different Genesets

In [ ]:
ALL_Genes_Corr = g_corr_df["Corr"].values

In [ ]:
ASD_GW = Fil2Dict("/home/jw3514/Work/UNIMED/dat/Genetics/GeneWeights/Rare/GW.ASD.top100.txt")
ASD_Genes = ASD_GW.keys()
ASD_Genes = [x for x in ASD_Genes if x in g_corr_df.index.values]
ASD_Genes_Corr = g_corr_df.loc[ASD_Genes, "Corr"].values

In [ ]:

SCZ_GW = Fil2Dict("/home/jw3514/Work/UNIMED/dat/Genetics/GeneWeights/Rare/GW.SCZ.top100.txt")
SCZ_Genes = SCZ_GW.keys()
SCZ_Genes = [x for x in SCZ_Genes if x in g_corr_df.index.values]
SCZ_Genes_Corr = g_corr_df.loc[SCZ_Genes, "Corr"].values

In [ ]:
Go2Genes = {}
fin = open("/home/jw3514/Work/UNIMED/notebooks/GOTerm_GeneSets.txt", "rt")
for l in fin.readlines():
    l = l.strip().split()
    GoName = l[0]
    Genes = [int(x) for x in l[1:]]
    Go2Genes[GoName] = Genes

In [ ]:
chromatin_binding = Go2Genes["GO:0003682-chromatin_binding"]
chromatin_binding = [x for x in chromatin_binding if x in g_corr_df.index.values]
chromatin_binding_Corr = g_corr_df.loc[chromatin_binding, "Corr"].values

In [ ]:
synapse = Go2Genes["GO:0045202-synapse"]
synapse = [x for x in synapse if x in g_corr_df.index.values]
synapse_Corr = g_corr_df.loc[synapse, "Corr"].values

In [ ]:
neuron_projection = Go2Genes["GO:0043005-neuron_projection"]
neuron_projection = [x for x in neuron_projection if x in g_corr_df.index.values]
neuron_projection_Corr = g_corr_df.loc[neuron_projection, "Corr"].values

In [ ]:
DNA_binding_transcription_factor_activity = Go2Genes["GO:0003700-DNA-binding_transcription_factor_activity"]
DNA_binding_transcription_factor_activity = [x for x in DNA_binding_transcription_factor_activity if x in g_corr_df.index.values]
DNA_binding_transcription_factor_activity_Corr = g_corr_df.loc[DNA_binding_transcription_factor_activity, "Corr"].values

In [ ]:
data = [ALL_Genes_Corr, ASD_Genes_Corr, SCZ_Genes_Corr, 
        chromatin_binding_Corr, DNA_binding_transcription_factor_activity_Corr, 
        synapse_Corr, neuron_projection_Corr]

# Create a violin plot
fig, ax = plt.subplots(dpi=120)
vp = ax.violinplot(data, [2, 4, 6, 8, 10, 12, 14], widths=1.5,
                   showmeans=True, showmedians=False, showextrema=False)

# Customize violin plot appearance
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink', 'lightcyan', 'lightgrey']
for patch, color in zip(vp['bodies'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor('black')
    patch.set_alpha(0.7)  # Set transparency

# Add grid lines
ax.grid(True, axis='y', linestyle='--', color='gray', alpha=0.5)

ax.set_xticks([2, 4, 6, 8, 10, 12, 14])  # Set tick positions
ax.set_xticklabels(['ALL_Genes', 'ASD_Genes', 'SCZ_Genes', 'Go:chromatin_binding', 
                    'Go:DNA_binding_TF_activity', 
                    'Go:synapse', 'Go:neuron_projection'], rotation=45)  # Set tick labels
ax.set_ylabel('Correlation')  

plt.show()

In [ ]:
## Add HighExp ISH / HighExp MERFISH
g_corr_df.head(2)

In [ ]:
for g, row in g_corr_df.iterrows():
    ish_exp_level = ISH_Exp.loc[g, :].mean()
    merfish_exp_level = MERFISH_Exp_Cell.loc[g, :].mean()
    g_corr_df.loc[g, "ISH_exp"] = ish_exp_level
    g_corr_df.loc[g, "MERFISH_exp"] = merfish_exp_level

In [ ]:
X = g_corr_df["Corr"]
Y = g_corr_df["ISH_exp"]
r, p = spearmanr(X,Y)
print(r)
plt.scatter(X,Y,s=1)

In [ ]:
X = g_corr_df["Corr"]
Y = g_corr_df["MERFISH_exp"]
r, p = spearmanr(X,Y)
print(r)
plt.scatter(X,Y,s=1)

In [ ]:
X = g_corr_df["ISH_exp"]
Y = g_corr_df["MERFISH_exp"]
r, p = spearmanr(X,Y)
print(r)
plt.scatter(X,Y,s=1)

In [ ]:
Median_ISH = np.median(g_corr_df["ISH_exp"].values)
Median_MERFISH = np.median(g_corr_df["MERFISH_exp"].values)

In [ ]:
Median_ISH

In [ ]:
#HighExp_ISH_Genes_Corr = g_corr_df[g_corr_df["ISH_exp"]>Median_ISH]["Corr"].values
#HighExp_MERFISH_Genes_Corr = g_corr_df[g_corr_df["MERFISH_exp"]>Median_MERFISH]["Corr"].values

HighExp_ISH_Genes_Corr = g_corr_df[g_corr_df["ISH_exp"]>1]["Corr"].values
HighExp_MERFISH_Genes_Corr = g_corr_df[g_corr_df["MERFISH_exp"]>1]["Corr"].values

In [ ]:
data = [ALL_Genes_Corr, ASD_Genes_Corr, SCZ_Genes_Corr, 
        chromatin_binding_Corr, DNA_binding_transcription_factor_activity_Corr, 
        synapse_Corr, neuron_projection_Corr,
        HighExp_ISH_Genes_Corr, HighExp_MERFISH_Genes_Corr]

# Create a violin plot
fig, ax = plt.subplots(dpi=120, figsize=(12,4))
vp = ax.violinplot(data, [2, 4, 6, 8, 10, 12, 14, 16, 18], widths=1.5,
                   showmeans=True, showmedians=False, showextrema=False)

# Customize violin plot appearance
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink', 'lightcyan', 'lightgrey']
##for patch, color in zip(vp['bodies'], colors):
#    patch.set_facecolor(color)
#    patch.set_edgecolor('black')
#    patch.set_alpha(0.7)  # Set transparency

for patch in vp['bodies']:
    patch.set_facecolor("lightcyan")
    patch.set_edgecolor('black')
    patch.set_alpha(0.7)  # Set transparency

# Add grid lines
ax.grid(True, axis='y', linestyle='--', color='gray', alpha=0.5)

ax.set_xticks([2, 4, 6, 8, 10, 12, 14, 16, 18])  # Set tick positions
ax.set_xticklabels(['ALL_Genes', 'ASD_Genes', 'SCZ_Genes', 'Go:chromatin_binding', 
                    'Go:DNA_binding_TF_activity', 
                    'Go:synapse', 'Go:neuron_projection', "HighEXP ISH", "HighEXP MERFISH"], rotation=45)  # Set tick labels
ax.set_ylabel('Correlation')  

plt.show()

In [ ]:
g_corr_df.shape

In [ ]:
g_corr_df_sort_exp = g_corr_df.sort_values("ISH_exp", ascending=False)

In [ ]:
corr_dat = []; exp_dat = []
for i in range(10):
    start = i * 1600
    end = (i+1) * 1600
    print(start, end)
    corr = g_corr_df_sort_exp["Corr"].values[start:end]
    mean_exp = g_corr_df_sort_exp["ISH_exp"].values[start:end].mean()
    corr_dat.append(corr); exp_dat.append(mean_exp)

In [ ]:
# Create a violin plot
fig, ax = plt.subplots(dpi=120, figsize=(12,4))
vp = ax.violinplot(corr_dat, [2, 4, 6, 8, 10, 12, 14, 16, 18, 20], widths=1.5,
                   showmeans=True, showmedians=False, showextrema=False)

for patch in vp['bodies']:
    patch.set_facecolor("lightcyan")
    patch.set_edgecolor('black')
    patch.set_alpha(0.7)  # Set transparency

# Add grid lines
ax.grid(True, axis='y', linestyle='--', color='gray', alpha=0.5)

ax.set_xticks([2, 4, 6, 8, 10, 12, 14, 16, 18, 20])  # Set tick positions
ax.set_xticklabels(["top {}0-{}0% ".format(i, i+1) for i in range(10)], rotation=45)  # Set tick labels
ax.set_ylabel('Correlation')  
ax.set_ylim(-0.75, 1)
plt.show()

In [ ]:
neuron_projection_DF = g_corr_df.loc[neuron_projection, :]
neuron_projection_DF = neuron_projection_DF.sort_values("ISH_exp", ascending=False)
neuron_projection_DF.shape

In [ ]:
dat_np = []
exp_np = []
for i in range(10):
    start = i * 27
    end = (i+1) * 27
    print(start, end)
    corr = neuron_projection_DF["Corr"].values[start:end]
    mean_exp = neuron_projection_DF["ISH_exp"].values[start:end].mean()
    dat_np.append(corr); exp_np.append(mean_exp)

In [ ]:
fig, ax = plt.subplots(dpi=120, figsize=(4,4))
ax.plot([2, 4, 6, 8, 10, 12, 14, 16, 18, 20], exp_np, marker="o", label="Neuro_projection")
ax.plot([2, 4, 6, 8, 10, 12, 14, 16, 18, 20], exp_dat, marker="o", label="All Gene")
plt.grid(True)
plt.legend()
ax.set_xticks([2, 4, 6, 8, 10, 12, 14, 16, 18, 20])  # Set tick positions
ax.set_xticklabels(["top {}0-{}0% ".format(i, i+1) for i in range(10)], rotation=45)  # Set tick labels
ax.set_xlabel("Exp Decile")
ax.set_ylabel("Exp Level (ISH)")
plt.show()

In [ ]:
# Create a violin plot
fig, ax = plt.subplots(dpi=120, figsize=(12,4))
vp = ax.violinplot(dat, [2, 4, 6, 8, 10, 12, 14, 16, 18, 20], widths=1.5,
                   showmeans=True, showmedians=False, showextrema=False)

for patch in vp['bodies']:
    patch.set_facecolor("lightcyan")
    patch.set_edgecolor('black')
    patch.set_alpha(0.7)  # Set transparency

# Add grid lines
ax.grid(True, axis='y', linestyle='--', color='gray', alpha=0.5)

ax.set_xticks([2, 4, 6, 8, 10, 12, 14, 16, 18, 20])  # Set tick positions
ax.set_xticklabels(["top {}0-{}0% ".format(i, i+1) for i in range(10)], rotation=45)  # Set tick labels
ax.set_ylabel('Correlation')  
ax.set_ylim(-0.75, 1)
plt.show()

### End

### Check individual STR which STR has best corr

In [ ]:
STR_corr = []
for STR in Shared_STRs:
    dat1 = ISH_Exp.loc[Shared_Genes, STR]
    dat2 = MERFISH_Exp_Cell.loc[Shared_Genes, STR]
    r, p = spearmanr(dat1, dat2, nan_policy="omit")
    STR_corr.append(r)
STR_corr = np.array(STR_corr)

In [ ]:
sns.kdeplot(STR_corr)
plt.xlabel("Corr cross genes")
#sns.kdeplot(g_corr_new)

In [ ]:
STR_corr2 = []
for STR in Shared_STRs:
    dat1 = ISH_Exp_HighCorr.loc[HighlyCorrG.index.values, STR]
    dat2 = Merfish_Exp_HighCorr.loc[HighlyCorrG.index.values, STR]
    r, p = spearmanr(dat1, dat2, nan_policy="omit")
    STR_corr2.append(r)
STR_corr2 = np.array(STR_corr2)

In [ ]:
sns.kdeplot(STR_corr, label="all genes")
sns.kdeplot(STR_corr2, label="Highly STR corr genes")
plt.xlabel("Corr cross genes")
plt.legend()
#sns.kdeplot(g_corr_new)

### Expression Bias with only high corr ASD genes

In [ ]:
def MakeNewGW(OldGW, df):
    GW_new = {}
    for k,v in OldGW.items():
        if k in df.index.values:
            GW_new[k] = v
    return GW_new

def FixSubiculum(DF):
    X = DF.loc["Subiculum_dorsal_part"]
    Y = DF.loc["Subiculum_ventral_part"]
    Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
    DF.loc["Subiculum"] = Z
    DF = DF.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])
    return DF

In [ ]:
MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.csv",index_col=0)
MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv",index_col=0)
head = MERFISH_Z2_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Z2_Cell.columns = new_head
ISH_Z2 = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

In [ ]:
MERFISH_Z2_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_VolMean.Z2.csv",index_col=0)
head = MERFISH_Z2_Vol.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Z2_Vol.columns = new_head

In [ ]:
ASD_GW = Fil2Dict("/home/jw3514/Work/UNIMED/dat/Genetics/GeneWeights/Rare/GW.ASD.top60.txt")
ASD_Genes = ASD_GW.keys()
ASD_Genes = [x for x in ASD_Genes if x in g_corr_df.index.values]

In [ ]:
len(ASD_GW)

In [ ]:
tmp.head(5)

In [ ]:
#Different Gene subgroups
tmp = g_corr_df.loc[ASD_Genes, :]
DF_highCorr = tmp[tmp["Corr"]>0.3]; 
GW_highCorr = MakeNewGW(ASD_GW, DF_highCorr)

DF_highISHExp = tmp[tmp["ISH_exp"]>1.5]; 
GW_highISHExp = MakeNewGW(ASD_GW, DF_highISHExp)

DF_highSCExp = tmp[tmp["MERFISH_exp"]>5]; 
GW_highSCExp = MakeNewGW(ASD_GW, DF_highSCExp)

In [ ]:
print(len(GW_highCorr), len(GW_highISHExp), len(GW_highSCExp))

In [ ]:
ASD_ISH = AvgSTRZ_Weighted(ISH_Z2, ASD_GW) 
ASD_ISH = FixSubiculum(ASD_ISH)
ASD_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW) 
ASD_MERFISH2 = AvgSTRZ_Weighted(MERFISH_Z2_Vol, ASD_GW) 

In [ ]:
ASD_Hcorr_ISH = AvgSTRZ_Weighted(ISH_Z2, GW_highCorr) 
ASD_Hcorr_ISH = FixSubiculum(ASD_Hcorr_ISH)
ASD_Hcorr_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, GW_highCorr) 
ASD_Hcorr_MERFISH2 = AvgSTRZ_Weighted(MERFISH_Z2_Vol, GW_highCorr) 

In [ ]:
ASD_H_ISHExp_ISH = AvgSTRZ_Weighted(ISH_Z2, GW_highISHExp) 
ASD_H_ISHExp_ISH = FixSubiculum(ASD_H_ISHExp_ISH)
ASD_H_ISHExp_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, GW_highISHExp) 

In [ ]:
ASD_H_SCExp_ISH = AvgSTRZ_Weighted(ISH_Z2, GW_highSCExp) 
ASD_H_SCExp_ISH = FixSubiculum(ASD_H_SCExp_ISH)
ASD_H_SCExp_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, GW_highSCExp) 

In [ ]:
#ASD_H_SCExp_MERFISH.head(50)

In [ ]:
#ASD_Hcorr_MERFISH.head(50)
#ASD_Hcorr_MERFISH[ASD_Hcorr_MERFISH["REGION"]=="Hippocampus"]

In [ ]:
# BiasCorrelation(ASD_Hcorr_ISH, ASD_Hcorr_MERFISH, "High Corr ASD ISH", "High Corr ASD MERFISH")
print(GetBiasCorrelation_STR(ASD_Hcorr_ISH, ASD_Hcorr_MERFISH))
#print(GetBiasCorrelation_STR(ASD_Hcorr_ISH, ASD_Hcorr_MERFISH2))
print(GetBiasCorrelation_STR(ASD_Hcorr_ISH, ASD_ISH))
print(GetBiasCorrelation_STR(ASD_Hcorr_ISH, ASD_MERFISH))
print(GetBiasCorrelation_STR(ASD_Hcorr_MERFISH, ASD_ISH))
#print(GetBiasCorrelation_STR(ASD_Hcorr_MERFISH2, ASD_ISH))

In [ ]:
print(GetBiasCorrelation_STR(ASD_H_ISHExp_ISH, ASD_H_ISHExp_MERFISH))
print(GetBiasCorrelation_STR(ASD_H_ISHExp_ISH, ASD_ISH))
print(GetBiasCorrelation_STR(ASD_H_ISHExp_MERFISH, ASD_MERFISH))
print(GetBiasCorrelation_STR(ASD_H_ISHExp_MERFISH, ASD_ISH))

In [ ]:
print(GetBiasCorrelation_STR(ASD_H_SCExp_ISH, ASD_H_SCExp_MERFISH))
print(GetBiasCorrelation_STR(ASD_H_SCExp_ISH, ASD_ISH))
print(GetBiasCorrelation_STR(ASD_H_SCExp_MERFISH, ASD_MERFISH))
print(GetBiasCorrelation_STR(ASD_H_SCExp_MERFISH, ASD_ISH))

In [ ]:
#ASD_Genes
tmp = g_corr_df.loc[ASD_Genes, :]
tmp2 = tmp[tmp["Corr"]<0.25]
tmp2.shape

In [ ]:
ASD_GW_new2 = {}
for k,v in ASD_GW.items():
    if k in tmp2.index.values:
        ASD_GW_new2[k] = v

In [ ]:
ASD_Lcorr_ISH = AvgSTRZ_Weighted(ISH_Z2, ASD_GW_new2) 
ASD_Lcorr_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW_new2) 

In [ ]:
X = ASD_Lcorr_ISH.loc["Subiculum_dorsal_part"]
Y = ASD_Lcorr_ISH.loc["Subiculum_ventral_part"]
Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
ASD_Lcorr_ISH.loc["Subiculum"] = Z
ASD_Lcorr_ISH = ASD_Lcorr_ISH.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])

In [ ]:
BiasCorrelation(ASD_Lcorr_ISH, ASD_Lcorr_MERFISH, "Low Corr ASD ISH", "Low Corr ASD MERFISH")

In [ ]:
ASD_Lcorr_ISH.head(50)

In [ ]:
ASD_Lcorr_MERFISH.head(50)

#### Test

In [ ]:
HY_STRs = ASD_ISH[ASD_ISH["REGION"]=="Hypothalamus"].index.values
CTX_STRs = ASD_ISH[ASD_ISH["REGION"]=="Isocortex"].index.values

In [ ]:
tmp.head(2)

In [ ]:
for g, row in tmp.iterrows():
    HY_Bias_SC = MERFISH_Z2_Cell.loc[g, HY_STRs].mean()
    CTX_Bias_SC = MERFISH_Z2_Cell.loc[g, CTX_STRs].mean()
    tmp.loc[g, "HY_Bias_SC"] = HY_Bias_SC
    tmp.loc[g, "CTX_Bias_SC"] = CTX_Bias_SC
    HY_Bias_ISH = ISH_Z2.loc[g, HY_STRs].mean()
    CTX_Bias_ISH = ISH_Z2.loc[g, CTX_STRs].mean()
    tmp.loc[g, "HY_Bias_ISH"] = HY_Bias_ISH
    tmp.loc[g, "CTX_Bias_ISH"] = CTX_Bias_ISH

In [ ]:
tmp = tmp.sort_values("HY_Bias_SC", ascending=False)

In [ ]:
tmp

In [ ]:
tmp[tmp["Symbol"]=="DDX3X"]

In [ ]:
HY = ASD_Lcorr_MERFISH[ASD_Lcorr_MERFISH["REGION"]=="Hypothalamus"].index.values
CTX = ASD_Lcorr_MERFISH[ASD_Lcorr_MERFISH["REGION"]=="Isocortex"].index.values

In [ ]:
HumanCT_ExpL = pd.read_csv("/home/jw3514/Work/CellType_Psy/dat2/HumanCT.ExpL.log.csv", index_col=0)
HumanCT_ExpL.columns = [int(x) for x in HumanCT_ExpL.columns.values]

In [ ]:
#GeneID = 51317
#GeneID = 57680
#GeneID = 2904
GeneID = 85358

X = ISH_Exp.loc[GeneID, Shared_STRs].values
Y = MERFISH_Exp_Cell.loc[GeneID, Shared_STRs].values

X_hy = ISH_Exp.loc[GeneID, HY].values
Y_hy = MERFISH_Exp_Cell.loc[GeneID, HY].values

X_ctx = ISH_Exp.loc[GeneID, CTX].values
Y_ctx = MERFISH_Exp_Cell.loc[GeneID, CTX].values

X_hy_human = HumanCT_ExpL.loc[GeneID, Human_HY].values
X_ctx_human = HumanCT_ExpL.loc[GeneID, Human_CTX].values


In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,6), dpi=480)

#plt.title(GeneID)
ax[0,0].scatter(X,Y, color="grey")
ax[0,0].scatter(X_hy,Y_hy, color="red", label="HY")
ax[0,0].scatter(X_ctx,Y_ctx, color="blue", label="CTX")
ax[0,0].set_xlabel("ISH")
ax[0,0].set_ylabel("SC")
ax[0,0].legend()
ax[0,0].set_title("{} {}".format(GeneID, Entrez2Symbol.get(GeneID, ".")))
#plt.title("ExpL of {}".format("PHF21A"))

ax[0,1].hist(X_hy_human, label="HY", color="red", bins=20, alpha=0.5)
ax[0,1].hist(X_ctx_human, label="IT", color="blue", bins=20, alpha=0.5)
ax[0,1].legend()
ax[0,1].set_title("Human CT")

ax[1,0].hist(X_hy, label="HY", color="red", bins=20, alpha=0.5)
ax[1,0].hist(X_ctx, label="CTX", color="blue", bins=20, alpha=0.5)
ax[1,0].legend()
ax[1,0].set_title("Mouse ISH")

ax[1,1].hist(Y_hy, label="HY", color="red", bins=20, alpha=0.5)
ax[1,1].hist(Y_ctx, label="CTX", color="blue", bins=20, alpha=0.5)
ax[1,1].legend()
ax[1,1].set_title("Mouse SC")

plt.tight_layout()

plt.savefig("figs/GeneCorrelation/{}.pdf".format(GeneID))

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("output.pdf")
for GeneID, row in tmp.iterrows():
    print(GeneID)
    X = ISH_Exp.loc[GeneID, Shared_STRs].values
    Y = MERFISH_Exp_Cell.loc[GeneID, Shared_STRs].values

    X_hy = ISH_Exp.loc[GeneID, HY].values
    Y_hy = MERFISH_Exp_Cell.loc[GeneID, HY].values

    X_ctx = ISH_Exp.loc[GeneID, CTX].values
    Y_ctx = MERFISH_Exp_Cell.loc[GeneID, CTX].values

    X_hy_human = HumanCT_ExpL.loc[GeneID, Human_HY].values
    X_ctx_human = HumanCT_ExpL.loc[GeneID, Human_CTX].values
    
    fig, ax = plt.subplots(2,2, figsize=(8,6), dpi=480)

    #plt.title(GeneID)
    ax[0,0].scatter(X,Y, color="grey")
    ax[0,0].scatter(X_hy,Y_hy, color="red", label="HY")
    ax[0,0].scatter(X_ctx,Y_ctx, color="blue", label="CTX")
    ax[0,0].set_xlabel("ISH")
    ax[0,0].set_ylabel("SC")
    ax[0,0].legend()
    ax[0,0].set_title("{} {}".format(GeneID, Entrez2Symbol.get(GeneID, ".")))
    #plt.title("ExpL of {}".format("PHF21A"))

    ax[0,1].hist(X_hy_human, label="HY", color="red", bins=20, alpha=0.5)
    ax[0,1].hist(X_ctx_human, label="IT", color="blue", bins=20, alpha=0.5)
    ax[0,1].legend()
    ax[0,1].set_title("Human CT")

    ax[1,0].hist(X_hy, label="HY", color="red", bins=20, alpha=0.5)
    ax[1,0].hist(X_ctx, label="CTX", color="blue", bins=20, alpha=0.5)
    ax[1,0].legend()
    ax[1,0].set_title("Mouse ISH")

    ax[1,1].hist(Y_hy, label="HY", color="red", bins=20, alpha=0.5)
    ax[1,1].hist(Y_ctx, label="CTX", color="blue", bins=20, alpha=0.5)
    ax[1,1].legend()
    ax[1,1].set_title("Mouse SC")

    plt.tight_layout()

    
    
    pdf.savefig( fig )
pdf.close()

In [ ]:
No_Gene_Drop = []
Bias_Corr = []
for i in range(30):
    tmptmpdf = tmp.loc[tmp.index.values[i:], :]
    #print(tmptmpdf.shape)
    tmp_gw = MakeNewGW(ASD_GW, tmptmpdf)
    
    tmp_ASD_ISH = AvgSTRZ_Weighted(ISH_Z2, tmp_gw) 
    tmp_ASD_ISH = FixSubiculum(tmp_ASD_ISH)
    tmp_ASD_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, tmp_gw) 
    r1,r2 = GetBiasCorrelation_STR(tmp_ASD_ISH, tmp_ASD_MERFISH)
    #print(tmptmpdf.shape[0], r1)
    Bias_Corr.append(r1)
    No_Gene_Drop.append(i+1)

In [ ]:
plt.figure(dpi=120)
plt.plot(No_Gene_Drop, Bias_Corr, marker="o", label="remove highest HY biased genes")
plt.grid(True)
plt.xlabel("No Gene Removed")
plt.ylabel("STR Bias Correlation")
plt.legend()

In [ ]:
plt.scatter(tmp["HY_Bias_ISH"].values, tmp["HY_Bias_SC"].values)
plt.xlabel("HY_Bias_ISH")
plt.ylabel("HY_Bias_SC")

In [ ]:
#MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.csv",index_col=0)
MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv",index_col=0)
head = MERFISH_Z2_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Z2_Cell.columns = new_head
ISH_Z2 = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

In [ ]:
ISH_SC_CorrDF = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v2.csv", index_col=0)

In [ ]:
ASD_CorrDF = ISH_SC_CorrDF.loc[ASD_Genes, :]

In [ ]:
tmp2

In [ ]:
ASD_GW_adj = {}
ASD_GW_adj2 = {}
for k,v in ASD_GW.items():
    if k in tmp2.index.values:
        ASD_GW_adj[k] = v * (ISH_SC_CorrDF.loc[k, "V2_V3_CT_Corr"]**2)
        ASD_GW_adj2[k] = v * (ISH_SC_CorrDF.loc[k, "Corr"]**2)

In [ ]:
print(len(ASD_GW), len(ASD_GW_adj))

In [ ]:
ASD_MERFISH = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW) 
ASD_MERFISH_adj = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW_adj) 
ASD_MERFISH_adj2 = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW_adj2) 

In [ ]:
#ASD_MERFISH.head(50)

In [ ]:
BiasCorrelation(ASD_ISH, ASD_MERFISH, "ISH", "MERFISH")

In [ ]:
ASD_MERFISH_adj[ASD_MERFISH_adj["REGION"]=="Thalamus"]

In [ ]:
BiasCorrelation(ASD_ISH, ASD_MERFISH_adj, "ISH", "MERFISH adj")

In [ ]:
BiasCorrelation(ASD_ISH, ASD_MERFISH_adj2, "ISH", "MERFISH adj2")

In [ ]:
ASD_ISH.to_csv("dat/Bias/ASD.ISH.bias.csv")
ASD_MERFISH_adj.to_csv("dat/Bias/ASD.MERFISH.Agg.adj.bias.csv")

### Allen Mouse Single Cell Bias with stratified genes

#### ASD

In [ ]:
CellTypesDF = pd.read_csv("dat/CellTypeHierarchy.csv")
Class2Cluster = {}
Subclass2Cluster = {}
for i, row in CellTypesDF.iterrows():
    _cluster, _class, _subclass, _supertype = row
    if _class not in Class2Cluster:
        Class2Cluster[_class] = []
    if _subclass not in Subclass2Cluster:
        Subclass2Cluster[_subclass] = []
    Class2Cluster[_class].append(_cluster)
    Subclass2Cluster[_subclass].append(_cluster)

In [ ]:
V3_subclass_Z2_Mat = pd.read_csv("dat/Z2Mat.ABCMatch.subclass.10xV3.csv", index_col=0)
max_Z, min_Z = 5, -5
V3_subclass_Z2_Mat_clipped = V3_subclass_Z2_Mat.clip(upper=max_Z)
ClusterV3_Z2_Mat = pd.read_csv("dat/Z2Mat.ABCMatch.cluster.10xV3.csv", index_col=0)
max_Z, min_Z = 5, -5
ClusterV3_Z2_Mat_clipped = ClusterV3_Z2_Mat.clip(upper=max_Z)

In [ ]:
ASD_CT_Bias_Hcorr = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_GW_new)

In [ ]:
ASD_CT_Bias_Hcorr.head(2)

In [ ]:
ASD_CT_Bias_Hcorr_dat = []
Class = sorted(Class2Cluster.keys())
for _CT in Class:
   #X22q_dat_V2.append([x for x in X22q_Bias_ClusterV2.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])
    ASD_CT_Bias_Hcorr_dat.append([x for x in ASD_CT_Bias_Hcorr.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(ASD_CT_Bias_Hcorr_dat, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_Glut = Subclass[0:27]
asd_dat_CNU_HYa_Glut_CB = []
for _CT in CNU_HYa_Glut:
    asd_dat_CNU_HYa_Glut_CB.append([x for x in ASD_CT_Bias_Hcorr.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])
fig, ax = plt.subplots(dpi=120)
ax.boxplot(asd_dat_CNU_HYa_Glut_CB, labels=CNU_HYa_Glut, vert=False)
plt.xlabel("ASD Cluster Z2 bias (CB)")
ax.grid(True)
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
Subclass = sorted(Subclass2Cluster.keys())
GNU_LGE = Subclass[59:65]
asd_dat_GNU_LGE_CB = []
for _CT in GNU_LGE:
    asd_dat_GNU_LGE_CB.append([x for x in ASD_CT_Bias_Hcorr.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])
fig, ax = plt.subplots(dpi=480, figsize=(6,3))
ax.boxplot(asd_dat_GNU_LGE_CB, labels=GNU_LGE, vert=False)
plt.xlabel("ASD Cluster Z2 bias (V2)")
ax.grid(True)
#ax[1].grid(True)
plt.tight_layout()
plt.show()

In [ ]:
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_GABA = Subclass[73:91]
asd_dat_CNU_HYa_GABA_CB = []
for _CT in CNU_HYa_GABA:
    asd_dat_CNU_HYa_GABA_CB.append([x for x in ASD_CT_Bias_Hcorr.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])
fig, ax = plt.subplots(dpi=120)
ax.boxplot(asd_dat_CNU_HYa_GABA_CB, labels=CNU_HYa_GABA, vert=False)
plt.xlabel("ASD Cluster Z2 bias (CB)")
ax.grid(True)
ax.grid(True)
plt.tight_layout()
plt.show()

#### SCZ

In [ ]:
SCZ_GW = Fil2Dict("/home/jw3514/Work/UNIMED/dat/Genetics/GeneWeights/Rare/GW.SCZ.top1000.txt")
SCZ_Genes = SCZ_GW.keys()
SCZ_Genes = [x for x in SCZ_Genes if x in g_corr_df.index.values]
SCZ_Genes_Corr = g_corr_df.loc[SCZ_Genes, "Corr"].values

In [ ]:
tmp = g_corr_df.loc[SCZ_Genes, :]
tmp2 = tmp[tmp["Corr"]>0.25]
tmp2.shape

In [ ]:
SCZ_GW_new = {}
for k,v in SCZ_GW.items():
    if k in tmp2.index.values:
        SCZ_GW_new[k] = v

In [ ]:
print(len(SCZ_GW), len(SCZ_GW_new))

In [ ]:
SCZ_CT_Bias_Hcorr = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, SCZ_GW_new)

In [ ]:
SCZ_CT_Bias_Hcorr_dat = []
Class = sorted(Class2Cluster.keys())
for _CT in Class:
   #X22q_dat_V2.append([x for x in X22q_Bias_ClusterV2.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])
    SCZ_CT_Bias_Hcorr_dat.append([x for x in SCZ_CT_Bias_Hcorr.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(SCZ_CT_Bias_Hcorr_dat, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
SCZ_CT_Bias_ALL = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, SCZ_GW)

In [ ]:
SCZ_CT_Bias_ALL_dat = []
Class = sorted(Class2Cluster.keys())
for _CT in Class:
   #X22q_dat_V2.append([x for x in X22q_Bias_ClusterV2.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])
    SCZ_CT_Bias_ALL_dat.append([x for x in SCZ_CT_Bias_ALL.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(SCZ_CT_Bias_ALL_dat, labels=Class, vert=False)
plt.grid(True)
plt.show()

### Allen Mouse Single Cell Bias Correted by Siblings

In [ ]:
ASD_CT_Bias = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_GW)

In [ ]:
ASD_CT_Bias.to_csv("test.csv")

In [ ]:
ClusterAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")
def add_class(BiasDF, ClusterAnn):
    for cluster, row in BiasDF.iterrows():
        BiasDF.loc[cluster, "class_id_label"] = ClusterAnn.loc[cluster, "class_id_label"]
        BiasDF.loc[cluster, "CCF_broad.freq"] = ClusterAnn.loc[cluster, "CCF_broad.freq"]
        BiasDF.loc[cluster, "CCF_acronym.freq"] = ClusterAnn.loc[cluster, "CCF_acronym.freq"]
        BiasDF.loc[cluster, "v3.size"] = ClusterAnn.loc[cluster, "v3.size"]
        BiasDF.loc[cluster, "v2.size"] = ClusterAnn.loc[cluster, "v2.size"]
    return BiasDF

In [ ]:
ASD_CT_Bias = add_class(ASD_CT_Bias, ClusterAnn)

In [ ]:
ASD_CT_Bias.head(2)

In [ ]:
Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3"
data = []
CTs = ASD_CT_Bias.index.values
for i in range(1000):
    SibDF = pd.read_csv("{}/cont.bias.{}.csv".format(Dir, i), index_col=0)
    SibDF = SibDF.loc[CTs,:]
    biases = SibDF["EFFECT"].values
    data.append(biases)
data = np.array(data)
sib_ct_bias_df_clusterV3 = pd.DataFrame(data=data, columns=CTs)

In [ ]:
for ct, row in ASD_CT_Bias.iterrows():
    EFFECT = row["EFFECT"]
    mean_sib = sib_ct_bias_df_clusterV3[ct].mean()
    Effect2 = EFFECT - mean_sib
    ASD_CT_Bias.loc[ct, "EFFECT2"] = Effect2

In [ ]:
ASD_CT_Bias2 = ASD_CT_Bias.sort_values("EFFECT2", ascending=False)

ASD_CT_Bias2.rename(columns={'EFFECT2': 'EFFECT', 'EFFECT': 'EFFECT_old'}, inplace=True)
ASD_CT_Bias2["Rank"] = ASD_CT_Bias2['EFFECT'].rank(ascending=False)

In [ ]:
ASD_CT_Bias2.head(5)

In [ ]:
ASD_CT_Bias_sibcorrect_dat = []
ASD_CT_Bias_dat = []
Class = sorted(Class2Cluster.keys())
for _CT in Class:
    ASD_CT_Bias_sibcorrect_dat.append([x for x in ASD_CT_Bias.loc[Class2Cluster[_CT], "EFFECT2"].values if x==x])
    ASD_CT_Bias_dat.append([x for x in ASD_CT_Bias.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(ASD_CT_Bias_sibcorrect_dat, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(ASD_CT_Bias_dat, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
CompareCT_ABC(ASD_CT_Bias, ASD_CT_Bias2, "ASD ", "ASD sib correted")

In [ ]:
ASD_CT_Bias.head(2)

In [ ]:
##### Is hypothalamus CT over-represent in Siblings?
HY_GABA = ASD_CT_Bias[ASD_CT_Bias["class_id_label"]=="12 HY GABA"].index.values
IT_ET = ASD_CT_Bias[ASD_CT_Bias["class_id_label"]=="01 IT-ET Glut"].index.values

In [ ]:
Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3"
Sib_Isocortex = []
Sib_Hypothalamus = []
for i in range(1000):
    SibDF = pd.read_csv("{}/cont.bias.{}.csv".format(Dir, i), index_col=0)
    Sib_Isocortex.append(SibDF.loc[IT_ET, "EFFECT"].mean())
    Sib_Hypothalamus.append(SibDF.loc[HY_GABA, "EFFECT"].mean())

In [ ]:
plt.figure(dpi=120)
sns.kdeplot(Sib_Isocortex, label="Sib_Isocortex", color="blue", ls="--")
sns.kdeplot(Sib_Hypothalamus, label="Sib_Hypothalamus", color="red", ls="--")
plt.axvline(x=0, color="grey")
plt.legend()
plt.xlabel("Bias")
plt.title("SubSampledSib")

#### Subclass Level

In [ ]:
SubclassAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "subclass_annotation", index_col="subclass_id_label")
def SubClass_add_class(BiasDF, SubclassAnn=SubclassAnn):
    for sublcass, row in BiasDF.iterrows():
        BiasDF.loc[sublcass, "class_id_label"] = SubclassAnn.loc[sublcass, "class_id_label"]
    return BiasDF

In [ ]:
V3_subclass_Z2_Mat = pd.read_csv("dat/Z2Mat.ABCMatch.subclass.10xV3.csv", index_col=0)
max_Z, min_Z = 5, -5
V3_subclass_Z2_Mat_clipped = V3_subclass_Z2_Mat.clip(upper=max_Z)

In [ ]:
ASD_CT_Bias_Subclass = ABC_AvgCTZ_Weighted(V3_subclass_Z2_Mat_clipped, ASD_GW)
ASD_CT_Bias_Subclass = SubClass_add_class(ASD_CT_Bias_Subclass)

In [ ]:
ASD_CT_Bias_Subclass = ABC_AvgCTZ_Weighted(V3_subclass_Z2_Mat_clipped, ASD_GW_adj)
ASD_CT_Bias_Subclass = SubClass_add_class(ASD_CT_Bias_Subclass)

In [ ]:
ASD_CT_Bias_Subclass.head(2)

In [ ]:
ASD_dat_CB = []
Class = sorted(Class2Cluster.keys())
DF = ASD_CT_Bias_Subclass
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    ASD_dat_CB.append([x for x in subdf["EFFECT"].values if x==x])
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(ASD_dat_CB, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
## cluster level

In [ ]:
len(ASD_GW_adj)

In [ ]:
ClusterV3_Z2_Mat = pd.read_csv("dat/Z2Mat.ABCMatch.cluster.10xV3.csv", index_col=0)
max_Z, min_Z = 5, -5
ClusterV3_Z2_Mat_clipped = ClusterV3_Z2_Mat.clip(upper=max_Z, lower=min_Z)

In [ ]:
ASD_CT_Bias_Cluster = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_GW_adj)
ASD_CT_Bias_Cluster = add_class(ASD_CT_Bias_Cluster, ClusterAnn)

In [ ]:
dat_V3 = []
Class = sorted(Class2Cluster.keys())
for _CT in Class:
    dat_V3.append([x for x in ASD_CT_Bias_Cluster.loc[Class2Cluster[_CT], "EFFECT"].values if x==x])

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,8))
ax.boxplot(dat_V3, labels=Class, vert=False)
plt.grid(True)
plt.show()

In [ ]:
ASD_CT_Bias_Cluster.to_csv("dat/Bias/ASD.ClusterV3.DN.Z2.csv")